In [1]:
import numpy as np

In [2]:
data = np.load("data.npy")
np.random.default_rng(seed=42).shuffle(data)
data

array([[ -798, -2143,   315, ...,    97,   195,   230],
       [ 1027,  -854, -1321, ...,    28,   142,   229],
       [ 1549,  -973,   376, ...,   110,   129,   308],
       ...,
       [-1341, -1624,   450, ...,    96,   214,   301],
       [ -776, -1738,   -43, ...,    85,   195,   325],
       [  525, -1887,   765, ...,    95,   171,   187]], shape=(60000, 12))

In [3]:
y_data = data[:,-3:] - 180
print(y_data)
y_data = np.radians(y_data)
y_data

[[ -83   15   50]
 [-152  -38   49]
 [ -70  -51  128]
 ...
 [ -84   34  121]
 [ -95   15  145]
 [ -85   -9    7]]


array([[-1.44862328,  0.26179939,  0.87266463],
       [-2.65290046, -0.66322512,  0.85521133],
       [-1.22173048, -0.89011792,  2.23402144],
       ...,
       [-1.46607657,  0.59341195,  2.11184839],
       [-1.65806279,  0.26179939,  2.53072742],
       [-1.48352986, -0.15707963,  0.12217305]], shape=(60000, 3))

In [4]:
y_data_cos = np.cos(y_data)
y_data_cos

array([[ 0.12186934,  0.96592583,  0.64278761],
       [-0.88294759,  0.78801075,  0.65605903],
       [ 0.34202014,  0.62932039, -0.61566148],
       ...,
       [ 0.10452846,  0.82903757, -0.51503807],
       [-0.08715574,  0.96592583, -0.81915204],
       [ 0.08715574,  0.98768834,  0.99254615]], shape=(60000, 3))

In [5]:
y_data_sin = np.sin(y_data)
y_data_sin

array([[-0.99254615,  0.25881905,  0.76604444],
       [-0.46947156, -0.61566148,  0.75470958],
       [-0.93969262, -0.77714596,  0.78801075],
       ...,
       [-0.9945219 ,  0.5591929 ,  0.8571673 ],
       [-0.9961947 ,  0.25881905,  0.57357644],
       [-0.9961947 , -0.15643447,  0.12186934]], shape=(60000, 3))

In [6]:
y_data_transformed = np.concatenate((y_data_cos,y_data_sin), axis=1) # 3 cos then 3 sin
y_data_transformed

array([[ 0.12186934,  0.96592583,  0.64278761, -0.99254615,  0.25881905,
         0.76604444],
       [-0.88294759,  0.78801075,  0.65605903, -0.46947156, -0.61566148,
         0.75470958],
       [ 0.34202014,  0.62932039, -0.61566148, -0.93969262, -0.77714596,
         0.78801075],
       ...,
       [ 0.10452846,  0.82903757, -0.51503807, -0.9945219 ,  0.5591929 ,
         0.8571673 ],
       [-0.08715574,  0.96592583, -0.81915204, -0.9961947 ,  0.25881905,
         0.57357644],
       [ 0.08715574,  0.98768834,  0.99254615, -0.9961947 , -0.15643447,
         0.12186934]], shape=(60000, 6))

In [7]:
X_data = data[:, :-3]
X_data

array([[ -798, -2143,   315, ...,   212,   594,   -56],
       [ 1027,  -854, -1321, ...,     7,   591,   243],
       [ 1549,  -973,   376, ...,  -160,   486,     8],
       ...,
       [-1341, -1624,   450, ...,    92,   656,   -62],
       [ -776, -1738,   -43, ...,    15,   677,    35],
       [  525, -1887,   765, ...,   261,   539,    19]], shape=(60000, 9))

In [8]:
X_train = X_data[:50_000]
X_test = X_data[50_000:]
y_train = y_data_transformed[:50_000]
y_test = y_data_transformed[50_000:]
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(50000, 9) (10000, 9) (50000, 6) (10000, 6)


In [9]:
import tensorflow as tf
import tensorflow.keras as k
import keras_tuner as kt

def build_model(hp):
    n_layers = hp.Int("n_layers", min_value=1, max_value=5, default=2)
    n_neurons = hp.Int("n_neurons", min_value=16, max_value=512)
    lr = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2, sampling="log")

    model = k.Sequential()
    for _ in range(n_layers):
        model.add(k.layers.Dense(n_neurons, activation="relu"))
    model.add(k.layers.Dense(6, activation=None))
    model.compile(loss="mean_squared_error", optimizer="adam", metrics=["root_mean_squared_error"])

    return model

random_search_tuner = kt.RandomSearch(
    build_model, objective="val_loss", max_trials=20, overwrite=True,
    directory=".", project_name="random_search", seed=42)

early_stop_cb = k.callbacks.EarlyStopping(patience=4, restore_best_weights=True)

random_search_tuner.search(X_train, y_train, epochs=20, validation_split=0.2, callbacks=[early_stop_cb])

Trial 20 Complete [00h 01m 43s]
val_loss: 0.030186736956238747

Best val_loss So Far: 0.021016879007220268
Total elapsed time: 00h 42m 00s


In [10]:
best_3trials = random_search_tuner.oracle.get_best_trials(num_trials=5)
for trial in best_3trials:
    print(trial.summary(), end="\n\n")

Trial 17 summary
Hyperparameters:
n_layers: 5
n_neurons: 287
learning_rate: 0.0033137745378583987
Score: 0.021016879007220268
None

Trial 09 summary
Hyperparameters:
n_layers: 4
n_neurons: 468
learning_rate: 0.00044412650444693207
Score: 0.023373890668153763
None

Trial 12 summary
Hyperparameters:
n_layers: 5
n_neurons: 88
learning_rate: 0.0003491260272208854
Score: 0.02396300621330738
None

Trial 04 summary
Hyperparameters:
n_layers: 5
n_neurons: 60
learning_rate: 0.008547485565344062
Score: 0.025876544415950775
None

Trial 14 summary
Hyperparameters:
n_layers: 4
n_neurons: 115
learning_rate: 0.0038287047044982298
Score: 0.026305431500077248
None



In [11]:
best_model = random_search_tuner.get_best_models(num_models=1)[0]
best_model.summary()

C:\Users\ianvi\env\Lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 26 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (32, 287)                   │           2,870 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (32, 287)                   │          82,656 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (32, 287)                   │          82,656 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (32, 287)                   │          82,656 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (32, 287)                   │          82,656 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (32, 6)                     │           1,728 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 335,222 (1.28 MB)

 Trainable params: 335,222 (1.28 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
early_stopping_cb = k.callbacks.EarlyStopping(patience=4, restore_best_weights=True)
history = best_model.fit(X_train, y_train, epochs=20, callbacks=[early_stopping_cb])

Epoch 1/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 14s 7ms/step - loss: 0.0213 - root_mean_squared_error: 0.1459
Epoch 2/20
  18/1563 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - loss: 0.0157 - root_mean_squared_error: 0.1252

C:\Users\ianvi\env\Lib\site-packages\keras\src\callbacks\early_stopping.py:99: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,root_mean_squared_error
  current = self.get_monitor_value(logs)


1563/1563 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - loss: 0.0203 - root_mean_squared_error: 0.1426
Epoch 3/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - loss: 0.0195 - root_mean_squared_error: 0.1398
Epoch 4/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - loss: 0.0193 - root_mean_squared_error: 0.1391
Epoch 5/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - loss: 0.0185 - root_mean_squared_error: 0.1359
Epoch 6/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - loss: 0.0180 - root_mean_squared_error: 0.1340
Epoch 7/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - loss: 0.0177 - root_mean_squared_error: 0.1331
Epoch 8/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - loss: 0.0175 - root_mean_squared_error: 0.1321
Epoch 9/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - loss: 0.0170 - root_mean_squared_error: 0.1303
Epoch 10/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - loss: 0.0172 - root_mean_squared_error: 0.1310
Epoch 11/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - loss: 0.0168 - r

In [13]:
best_model.evaluate(X_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.0158 - root_mean_squared_error: 0.1256


[0.015785938128829002, 0.1256421059370041]

In [14]:
best_model.save("best_NN_angle_estimation.keras")